In [2]:
import duckdb
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [ ]:

# -------------------------------------------------------------------
# Paths – adjust as needed
# -------------------------------------------------------------------
TRIPS_PATH = "../data/processed/citibike/*/*/data.parquet"

con = duckdb.connect()

# -------------------------------------------------------------------
# Base CTEs: trips, pickups, dropoffs, dense station × day grid
# -------------------------------------------------------------------

base_sql = f"""
WITH trips AS (
    SELECT
        start_station_name AS start_station,
        end_station_name  AS end_station,
        date_trunc('day', started_at) AS day
    FROM read_parquet('{TRIPS_PATH}')
    WHERE started_at IS NOT NULL
),
-- All unique stations (from starts and ends)
stations AS (
    SELECT DISTINCT station FROM (
        SELECT start_station AS station FROM trips
        UNION
        SELECT end_station   AS station FROM trips
    )
),
-- Date bounds
bounds AS (
    SELECT
        min(day) AS min_day,
        max(day) AS max_day
    FROM trips
),
-- All days between min and max (dense)
days AS (
    SELECT
        min_day + (i || ' days')::INTERVAL AS day
    FROM bounds,
         range(date_diff('day', min_day, max_day) + 1) AS t(i)
),
-- Dense station × day grid
grid AS (
    SELECT
        s.station,
        d.day
    FROM stations s
    CROSS JOIN days d
),
-- Daily pickups per station
pickups AS (
    SELECT
        start_station AS station,
        day,
        COUNT(*) AS n_pickups
    FROM trips
    GROUP BY start_station, day
),
-- Daily dropoffs per station
dropoffs AS (
    SELECT
        end_station AS station,
        day,
        COUNT(*) AS n_dropoffs
    FROM trips
    GROUP BY end_station, day
),
-- Combine grid with pickups & dropoffs
daily_base AS (
    SELECT
        g.station AS start_station_name,
        g.day,
        COALESCE(p.n_pickups, 0)  AS n_pickups,
        COALESCE(d.n_dropoffs, 0) AS n_dropoffs,
        COALESCE(d.n_dropoffs, 0) - COALESCE(p.n_pickups, 0) AS net_flow
    FROM grid g
    LEFT JOIN pickups  p USING (station, day)
    LEFT JOIN dropoffs d USING (station, day)
),
-- Add time features
daily_with_time AS (
    SELECT
        start_station_name,
        day,
        n_pickups,
        n_dropoffs,
        net_flow,
        CAST(strftime(day, '%Y') AS INTEGER) AS year,
        CAST(strftime(day, '%m') AS INTEGER) AS month,
        CAST(strftime(day, '%d') AS INTEGER) AS day_of_month,
        CAST(strftime(day, '%w') AS INTEGER) AS day_of_week,   -- 0=Sunday,...,6=Saturday
        CASE
            WHEN CAST(strftime(day, '%w') AS INTEGER) IN (0, 6) THEN 1 ELSE 0
        END AS is_weekend
    FROM daily_base
),
-- Add lag & rolling features (by station)
daily_features AS (
    SELECT
        *,
        -- 1-day lag of net flow
        LAG(net_flow, 1) OVER (
            PARTITION BY start_station_name
            ORDER BY day
        ) AS net_flow_lag_1d,

        -- 7-day lag of net flow
        LAG(net_flow, 7) OVER (
            PARTITION BY start_station_name
            ORDER BY day
        ) AS net_flow_lag_7d,

        -- rolling mean net flow over last 7 days (excluding today)
        AVG(net_flow) OVER (
            PARTITION BY start_station_name
            ORDER BY day
            ROWS BETWEEN 7 PRECEDING AND 1 PRECEDING
        ) AS net_flow_mean_7d
    FROM daily_with_time
)
SELECT * FROM daily_features
ORDER BY start_station_name, day
"""

df_daily = con.execute(base_sql).fetchdf()

con.close()

print(df_daily.head())
print(df_daily.shape)


  start_station_name        day  n_pickups  n_dropoffs  net_flow  year  month  \
0   1 Ave & E 110 St 2022-12-28          0           0         0  2022     12   
1   1 Ave & E 110 St 2022-12-29          0           0         0  2022     12   
2   1 Ave & E 110 St 2022-12-30          0           0         0  2022     12   
3   1 Ave & E 110 St 2022-12-31          0           0         0  2022     12   
4   1 Ave & E 110 St 2023-01-01         31          32         1  2023      1   

   day_of_month  day_of_week  is_weekend  net_flow_lag_1d  net_flow_lag_7d  \
0            28            3           0             <NA>             <NA>   
1            29            4           0                0             <NA>   
2            30            5           0                0             <NA>   
3            31            6           1                0             <NA>   
4             1            0           1                0             <NA>   

   net_flow_mean_7d  
0               NaN  


In [4]:
lag_cols = ["net_flow_lag_1d", "net_flow_lag_7d", "net_flow_mean_7d"]
df_model = df_daily.dropna(subset=lag_cols).copy()

In [5]:
# df_model["start_station_name"] = df_model["start_station_name"].astype("category").cat.codes
train = df_model[df_model["day"] <  "2025-07-01"]
val   = df_model[(df_model["day"] >= "2025-07-01") & (df_model["day"] < "2025-10-01")]
test  = df_model[df_model["day"] >= "2025-10-01"]

print(len(train), len(val), len(test))

2129787 215556 72633


In [7]:


feature_cols = [
    "start_station_name",
    "day_of_week",
    "is_weekend",
    "month",
    "net_flow_lag_1d",
    "net_flow_lag_7d",
    "net_flow_mean_7d",
]

# X_train = train[feature_cols]
# y_train = train["net_flow"]

# X_val   = val[feature_cols]
# y_val   = val["net_flow"]

# X_test  = test[feature_cols]
# y_test  = test["net_flow"]

X_train = pd.get_dummies(train[feature_cols], columns=["start_station_name"], sparse=True).values
y_train = train["net_flow"]

X_val   = pd.get_dummies(val[feature_cols], columns=["start_station_name"], sparse=True).values
y_val   = val["net_flow"]

X_test  = pd.get_dummies(test[feature_cols], columns=["start_station_name"], sparse=True).values
y_test  = test["net_flow"]

/var/folders/6c/gmnjxh5x1hn48qy93ptqpz580000gn/T/ipykernel_24953/743469052.py:20: FutureWarning: Allowing arbitrary scalar fill_value in SparseDtype is deprecated. In a future version, the fill_value must be a valid value for the SparseDtype.subtype.
  X_train = pd.get_dummies(train[feature_cols], columns=["start_station_name"], sparse=True).values
/var/folders/6c/gmnjxh5x1hn48qy93ptqpz580000gn/T/ipykernel_24953/743469052.py:23: FutureWarning: Allowing arbitrary scalar fill_value in SparseDtype is deprecated. In a future version, the fill_value must be a valid value for the SparseDtype.subtype.
  X_val   = pd.get_dummies(val[feature_cols], columns=["start_station_name"], sparse=True).values
/var/folders/6c/gmnjxh5x1hn48qy93ptqpz580000gn/T/ipykernel_24953/743469052.py:26: FutureWarning: Allowing arbitrary scalar fill_value in SparseDtype is deprecated. In a future version, the fill_value must be a valid value for the SparseDtype.subtype.
  X_test  = pd.get_dummies(test[feature_cols], co

In [ ]:
from sklearn.ensemble import HistGradientBoostingRegressor
model = HistGradientBoostingRegressor(
    max_iter=100,
    learning_rate=0.1,
    max_depth=6,
    random_state=42,
    verbose=1,
    # categorical_features=["start_station_name"],
)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
rmse = root_mean_squared_error(y_test, y_pred)
print(f"Test MAE: {mae:.2f}")
print(f"Test RMSE: {rmse:.2f}")


In [24]:
import xgboost as xgb
from sklearn.metrics import root_mean_squared_error


dtrain = xgb.DMatrix(X_train, label=y_train)
dval   = xgb.DMatrix(X_val,   label=y_val)
dtest  = xgb.DMatrix(X_test,  label=y_test)

params = {
    "objective": "reg:squarederror",   # net_flow can be negative, so plain regression
    "eval_metric": "rmse",
    "max_depth": 8,
    "eta": 0.1,
    "subsample": 0.8,
    "colsample_bytree": 0.8,
}

evals = [(dtrain, "train"), (dval, "val")]

bst = xgb.train(
    params,
    dtrain,
    num_boost_round=100,
    evals=evals,
    early_stopping_rounds=50,
    verbose_eval=50,
)

y_pred = bst.predict(dtest)

mae  = mean_absolute_error(y_test, y_pred)
rmse = root_mean_squared_error(y_test, y_pred)
wmape = np.abs(y_pred - y_test).sum() / np.abs(y_test).sum()

print(f"Test MAE (net_flow):  {mae:.3f}")
print(f"Test RMSE (net_flow): {rmse:.3f}")
print(f"Test WMAPE:           {wmape:.3%}")

/Users/jesper/Documents/AXA/DSC/.conda/lib/python3.11/site-packages/xgboost/data.py:399: UserWarning: Sparse arrays from pandas are converted into dense.
  warnings.warn("Sparse arrays from pandas are converted into dense.")
/Users/jesper/Documents/AXA/DSC/.conda/lib/python3.11/site-packages/xgboost/data.py:399: UserWarning: Sparse arrays from pandas are converted into dense.
  warnings.warn("Sparse arrays from pandas are converted into dense.")
/Users/jesper/Documents/AXA/DSC/.conda/lib/python3.11/site-packages/xgboost/data.py:399: UserWarning: Sparse arrays from pandas are converted into dense.
  warnings.warn("Sparse arrays from pandas are converted into dense.")


[0]	train-rmse:6.31780	val-rmse:7.48673
[50]	train-rmse:5.91194	val-rmse:7.12684
[99]	train-rmse:5.86905	val-rmse:7.09979
Test MAE (net_flow):  4.091
Test RMSE (net_flow): 7.048
Test WMAPE:           97.111%


In [13]:
df_daily["net_flow"].median()

0.0